In [ ]:
!pip install python-igraph requests pandas networkx
!pip install git+https://github.com/bdemchak/py4cytoscape

In [1]:
import py4cytoscape as py4
import pandas as pd
import os
import uuid
from logging import getLogger, INFO
from concurrent_log_handler import ConcurrentRotatingFileHandler
from IPython.display import Image
from time import sleep
%config InlineBackend.figure_format = 'svg'

#### Clear all networks at the begining

In [2]:
py4.session.close_session(False)

{}

#### Import Co-Exp Data and Rename columns

In [3]:
results = pd.read_excel('input_data/co_exp_result.xlsx')
results = results.rename(columns={'gene1': 'source', 'gene2': 'target'})

#### Create network with co-exp data and give random network name

In [4]:
network_name = uuid.uuid4().hex
network_suid = py4.networks.create_network_from_data_frames(
    edges=results, 
    title= network_name,
    collection= network_name + '_collection'
)

{'networkSUID': 29135}

#### Run analyze command so your network will be one way directed

In [5]:
analyze = py4.commands.commands_post('analyzer/analyze')

{'networkTitle': '1f6423c99b1045c39ecfbf5e271ac9c4 (undirected)',
 'nodeCount': '197',
 'edgeCount': '1122',
 'avNeighbors': '11.86096256684492',
 'diameter': '8',
 'radius': '5',
 'avSpl': '3.271634753608188',
 'cc': '0.5249889291420875',
 'density': '0.06376861595077914',
 'heterogeneity': '0.8353654292197396',
 'centralization': '0.16378959604766055',
 'ncc': '4',
 'time': '0.043'}

#### Run mcode command with options (you can change network settings)

In [ ]:
mcode = py4.commands.commands_post('mcode cluster degreeCutoff=2 fluff=false fluffNodeDensityCutoff=0.1 haircut=true includeLoops=false kCore=2 maxDepthFromStart=100 network=current nodeScoreCutoff=0.2 scope=NETWORK')
sleep(5)

#### Mcode created clusters, we need big clusters which is node count bigger than 20 (you can change threshold)

In [7]:
big_clusters = [i for i in range(len(mcode['clusters'])) if len(mcode['clusters'][i]['nodes']) >= 20]

[0, 1, 2]

#### Create empty list and append cluster tables to in it
#### Create cluster images as a png and network file as a .cyjs so you are able to use in cytoscape.js

In [8]:
clusters = []
for i in range(len(big_clusters)):
    view_id = py4.commands.commands_post('mcode view id=1 rank=' + str(i+1) )
    clusters.append( py4.tables.get_table_columns() \
                  .drop(columns=['id','name','selected','MCODE_Cluster','SUID']) \
                  .sort_values(by=['MCODE_Score'], ascending=False) \
                 .reset_index(drop=True) )
    
    py4.network_views.get_network_view_suid()
    img_name = uuid.uuid4().hex
    py4.export_image(str(img_name))
    py4.export_network(img_name,'cyjs')
    Image(img_name+'.png')